# 現代程式語言 期中專案 - 家樂福衛生紙價格比較
第八組 111210518 陳辰鈺、111210520 吳佳泰、111210524藍健洲  
date:10/28

## Step1.爬取資料

In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3


list1 = []
list2 = []

url = "https://online.carrefour.com.tw/zh/search/?q=%E6%8A%BD%E5%8F%96%20%E8%A1%9B%E7%94%9F%E7%B4%99"
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/80.0.3897.132 Safari/537.6"}

html = requests.get(url , headers=headers)
html.encoding = "UTF-8"

sp = BeautifulSoup(html.text , "lxml")

data1 = sp.find_all("div" , class_="commodity-desc")  ##name
data2 = sp.find_all("em")  ## price



## Step2.創建一個資料庫(tissue.db)，並將資料匯入

In [ ]:
counter = 1

for i in data1:
    if (i.find("a" , title=True).text == "家樂福抽取式衛生紙FSC" and counter == 1):
        list1.append((i.find("a" , title=True).text) + "24入")
        counter += 1
    elif (i.find("a" , title=True).text == "家樂福抽取式衛生紙FSC" and counter == 2):
        list1.append((i.find("a" , title=True).text) + "72入")
    else:
        list1.append((i.find("a" , title=True).text))

for i in data2:
    temp1 = list(i.text)
    
    for j in temp1:
        if (j == ","):
            temp1.remove(",")
        elif (j == "$"):
            temp1.remove("$")
    
    temp2 = "".join(temp1)
    temp2 = int(temp2)

    list2.append(temp2)

db = sqlite3.connect("tissue.db")
cursor = db.cursor()
cursor.execute(
    '''
    CREATE TABLE TISSUE(
    name    CHAR,
    price   INT
    )
    '''
)

for i in range (0 , len(list1)):
    cursor.execute("INSERT INTO TISSUE(name , price)  VALUES (?  , ?)" , (list1[i] , list2[i]))

db.commit()
db.close()